# Capstone Project

## Coursera Training: Applied Data Science Capstone

In [1]:
import pandas as pd
import numpy as np
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


### WEEK 3 
#### Exercise 1
Scrapper from: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

### Clean the Data

In [2]:
from bs4 import BeautifulSoup
import requests
CLIENT_ID='UBMB4ZEUX3ZCCXPPS2T2QG5VYJLWRKDW0O5AZ3OLFBZS3KTN'
CLIENT_SECRET='NYTCBMAPFYCV0W5WF1RQYBFWYBZ5LPGENMGINUCDIEGPC4B0'
LIMIT=100
VERSION=20180605
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


r  = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
html_soup = BeautifulSoup(r.text, 'html.parser')

l = []
for tr in html_soup.find_all('table')[0].find_all('tr'):
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    if len(tr.text)>0:
        l.append(row)

df_orig = pd.DataFrame(l)



#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

df_orig=df_orig.rename(columns={0:"PostalCode", 1:"Borough",2:"Neighborhood"})
df_orig = df_orig.replace(r'\n','', regex=True) 
df_orig.drop([0],axis=0,inplace=True)


#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df_orig = df_orig[df_orig['Borough'] != "Not assigned"]
df_orig


,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


### Data Frame Preparation

In [3]:
#More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, 
#you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
#These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in 
#the above table.

serie = df_orig.groupby(['PostalCode','Borough'])['Neighborhood'].apply(list)

df_format=pd.DataFrame(serie)
df_format['Neighborhood'] = df_format['Neighborhood'].apply(lambda x: ','.join(map(str, x)))
df_format=df_format.reset_index()
df_format.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Here is the shape of hte Data Frame

In [4]:
print(df_format.shape)


(103, 3)


#### Exercise 2

In [5]:
data_df=pd.read_csv('Geospatial_Coordinates.csv')
data_df.head()
join_data=data_df.set_index('Postal Code').join(df_orig.set_index('PostalCode'))
#join_data=join_data.reset_index()

join_data=join_data.rename(columns={0:"PostalCode", 1: "Latitude", 2: "Longitude", 3:"Borough",4:"Neighborhood"})
print(join_data.columns)
join_data.head(5)

Index(['Latitude', 'Longitude', 'Borough', 'Neighborhood'], dtype='object')


,Latitude,Longitude,Borough,Neighborhood
M1B,43.806686,-79.194353,Scarborough,Rouge
M1B,43.806686,-79.194353,Scarborough,Malvern
M1C,43.784535,-79.160497,Scarborough,Highland Creek
M1C,43.784535,-79.160497,Scarborough,Rouge Hill
M1C,43.784535,-79.160497,Scarborough,Port Union


### Toronto Data

In [6]:
df_toronto = join_data[join_data['Borough'].str.contains("Toronto")]
df_toronto

,Latitude,Longitude,Borough,Neighborhood
M4E,43.676357,-79.293031,East Toronto,The Beaches
M4K,43.679557,-79.352188,East Toronto,The Danforth West
M4K,43.679557,-79.352188,East Toronto,Riverdale
M4L,43.668999,-79.315572,East Toronto,The Beaches West
M4L,43.668999,-79.315572,East Toronto,India Bazaar
M4M,43.659526,-79.340923,East Toronto,Studio District
M4N,43.728020,-79.388790,Central Toronto,Lawrence Park
M4P,43.712751,-79.390197,Central Toronto,Davisville North
M4R,43.715383,-79.405678,Central Toronto,North Toronto West
M4S,43.704324,-79.388790,Central Toronto,Davisville


### Graphical Representation

In [7]:
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from sklearn.cluster import KMeans 
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [8]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [9]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(join_data['Borough'].unique()),
        join_data.shape[0]
    )
)



The dataframe has 11 boroughs and 211 neighborhoods.


In [10]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(join_data['Latitude'], join_data['Longitude'], join_data['Borough'], join_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### KNN

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
toronto_venues = getNearbyVenues(names=join_data['Neighborhood'],
                                   latitudes=join_data['Latitude'],
                                   longitudes=join_data['Longitude']
                                  )



Rouge
Malvern
Highland Creek
Rouge Hill
Port Union
Guildwood
Morningside
West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park
Ionview
Kennedy Park
Clairlea
Golden Mile
Oakridge
Cliffcrest
Cliffside
Scarborough Village West
Birch Cliff
Cliffside West
Dorset Park
Scarborough Town Centre
Wexford Heights
Maryvale
Wexford
Agincourt
Clarks Corners
Sullivan
Tam O'Shanter
Agincourt North
L'Amoreaux East
Milliken
Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview
Henry Farm
Oriole
Bayview Village
Silver Hills
York Mills
Newtonbrook
Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park
Don Mills South
Bathurst Manor
Downsview North
Wilson Heights
Northwood Park
York University
CFB Toronto
Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens
Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West
Riverdale
The Beaches West
Indi

In [18]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Adelaide,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.020000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.0
1,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,Agincourt North,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,Albion Gardens,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,Alderwood,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
5,Bathurst Manor,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.0
6,Bathurst Quay,0.000000,0.0,0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,...,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
7,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
8,Beaumond Heights,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
9,Bedford Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [19]:

# set number of clusters
kclusters = 5

manhattan_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

In [20]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Bar,Steakhouse,American Restaurant,Hotel,Restaurant,Burger Joint,Thai Restaurant,Cosmetics Shop
1,Agincourt,Skating Rink,Lounge,Breakfast Spot,Clothing Store,Women's Store,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
2,Agincourt North,Playground,Park,Women's Store,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
3,Albion Gardens,Grocery Store,Pharmacy,Coffee Shop,Beer Store,Sandwich Place,Fried Chicken Joint,Fast Food Restaurant,Pizza Place,Construction & Landscaping,Discount Store
4,Alderwood,Pizza Place,Pharmacy,Sandwich Place,Athletics & Sports,Pub,Pool,Skating Rink,Coffee Shop,Gym,Dessert Shop


In [21]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = join_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Latitude,Longitude,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
M1B,43.806686,-79.194353,Scarborough,Rouge,0.0,Fast Food Restaurant,Women's Store,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
M1B,43.806686,-79.194353,Scarborough,Malvern,0.0,Fast Food Restaurant,Women's Store,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
M1C,43.784535,-79.160497,Scarborough,Highland Creek,0.0,Bar,Moving Target,Women's Store,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
M1C,43.784535,-79.160497,Scarborough,Rouge Hill,0.0,Bar,Moving Target,Women's Store,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
M1C,43.784535,-79.160497,Scarborough,Port Union,0.0,Bar,Moving Target,Women's Store,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store


In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


tor=toronto_merged[toronto_merged['Cluster Labels'].isnull()]

print(tor)

      Latitude  Longitude      Borough      Neighborhood  Cluster Labels  \
M1X  43.836125 -79.205636  Scarborough       Upper Rouge             NaN   
M2L  43.757490 -79.374714   North York      Silver Hills             NaN   
M2L  43.757490 -79.374714   North York        York Mills             NaN   
M9A  43.667856 -79.532242    Etobicoke  Islington Avenue             NaN   

    1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
M1X                   NaN                   NaN                   NaN   
M2L                   NaN                   NaN                   NaN   
M2L                   NaN                   NaN                   NaN   
M9A                   NaN                   NaN                   NaN   

    4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
M1X                   NaN                   NaN                   NaN   
M2L                   NaN                   NaN                   NaN   
M2L                   NaN         

In [41]:

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if cluster > 0:
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[int(cluster)-1],
            fill=True,
            fill_color=rainbow[int(cluster)-1],
            fill_opacity=0.7).add_to(map_clusters)

map_clusters